In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from functools import reduce

In [2]:
df_static = pd.read_csv('pred_mor_static.csv')

df_static.drop('Unnamed: 0', axis=1, inplace=True)
df_static.head()

,RecordID,Age,Gender,Height,ICUType,Weight
0,132592.0,35.0,0.0,169.9,3.0,71.8
1,140662.0,42.0,1.0,169.9,3.0,138.1
2,140104.0,61.0,1.0,188.0,2.0,80.0
3,134685.0,53.0,0.0,170.2,4.0,96.1
4,134849.0,46.0,1.0,169.9,3.0,82.0


In [3]:
print(len(df_static['RecordID'].unique()))

4000


In [4]:
df_time = pd.read_csv('pred_mor_time.csv')
df_time.drop('Unnamed: 0', axis=1, inplace=True)

In [5]:
df_time.index[df_time['Parameter'] == 'ICUType'].tolist()

[6665,
 40232,
 72039,
 86042,
 135093,
 212847,
 262537,
 392663,
 418567,
 438694,
 467905,
 468914,
 482840,
 492957,
 495514,
 568946,
 572825,
 625918,
 725335,
 741944,
 763767,
 839454,
 843277,
 966568,
 992748,
 1009258,
 1038761,
 1057756,
 1170646,
 1280722,
 1330015,
 1468419,
 1471287,
 1527730,
 1536990,
 1629602,
 1645757,
 1702902,
 1704320,
 1729419]

In [6]:
df_time = df_time[df_time.Parameter != 'Height']
df_time = df_time[df_time.Parameter != 'Gender']
df_time = df_time[df_time.Parameter != 'ICUType']

In [7]:
df_static[~df_static.RecordID.isin(df_time.RecordID)]

,RecordID,Age,Gender,Height,ICUType,Weight
183,140501.0,31.0,1.0,169.9,3.0,82.0
649,140936.0,49.0,1.0,169.9,3.0,82.0
1991,141264.0,90.0,0.0,169.9,3.0,82.0


In [8]:
dftime_order = df_time.sort_values(by=['RecordID', 'Time'])

In [9]:
print(len(dftime_order['RecordID'].unique()))

3997


In [10]:
dfstatic_order = df_static.sort_values(by=['RecordID'])

In [11]:
row_index = sorted(df_time['RecordID'].unique())
print(len(row_index))

3997


In [12]:
params = sorted(df_time['Parameter'].unique())
params

['ALP',
 'ALT',
 'AST',
 'Albumin',
 'BUN',
 'Bilirubin',
 'Cholesterol',
 'Creatinine',
 'DiasABP',
 'FiO2',
 'GCS',
 'Glucose',
 'HCO3',
 'HCT',
 'HR',
 'K',
 'Lactate',
 'MAP',
 'MechVent',
 'Mg',
 'NIDiasABP',
 'NIMAP',
 'NISysABP',
 'Na',
 'PaCO2',
 'PaO2',
 'Platelets',
 'RespRate',
 'SaO2',
 'SysABP',
 'Temp',
 'TroponinI',
 'TroponinT',
 'Urine',
 'WBC',
 'Weight',
 'pH']

In [13]:
def extract_var(df, params, indexes):
    df = df.copy()
    
    # create empty lists
    col_headers = []
    dflst = []
    datalst = []
    dfss = []
    
    for x in params:
        col_head = []
        # create new df containing only this specific param
        new_df = df[df.iloc[:, 1] == x]
        # append list with new df
        dflst.append(new_df)
        
        # append col_head list with headers for stats for this param
        col_head.append('RecordID')
        col_head.append('{}_first'.format(x))
        col_head.append('{}_last'.format(x))
        col_head.append('{}_min'.format(x))
        col_head.append('{}_max'.format(x))
        col_head.append('{}_q1'.format(x))
        col_head.append('{}_median'.format(x))
        col_head.append('{}_q3'.format(x))
        col_head.append('{}_mean'.format(x))
        col_head.append('{}_count'.format(x))
        
        col_headers.append(col_head)
    
    # loop through dflst to get stats for each df
    for y in dflst:
        # make list where each ID occurs only once
        uni_ids = y['RecordID'].unique()
        
        # create list to be filled with stats lists
        lst = []
        # for each recordID/patient, get the first, last, max, min
            # mean, median, q1, q2, count values where possible
        for z in uni_ids:
            # create list to be filled with stats
            idl = []
            
            # calc stats values
            patient = y[y['RecordID'] == z]   
            first = patient['Time'].idxmin
            last = patient['Time'].idxmax
            minv = patient['Value'].min()
            maxv = patient['Value'].max()
            q1 = patient['Value'].quantile(0.25)
            median = patient['Value'].quantile(0.5)
            q3 = patient['Value'].quantile(0.75)
            mean = patient['Value'].mean()
            count = patient['Value'].count()
            iqr = q3 - q1

            # append list with descriptive stats
            idl.append(z)
            idl.append(patient['Value'][first])
            idl.append(patient['Value'][last])
            if minv < (q1 - (1.5*iqr)):
                idl.append(q1 - 0.5*iqr)
            else:
                idl.append(minv)
            if maxv > (q3 + (1.5*iqr)):
                idl.append(q3 + 0.5*iqr)
            else:
                idl.append(maxv)
            idl.append(q1)
            idl.append(median)
            idl.append(q3)
            idl.append(mean)
            idl.append(count)
            # append lst with list of stats
            lst.append(idl)
        # append datalst with list of lists of stats for each unique ID    
        datalst.append(lst)
    
    # use enumerate to make use of indexes and create new dfs of stats
        # with headers
    for u,v in enumerate(col_headers):
        stats = np.array(datalst[u])
        new_df = pd.DataFrame(data=stats, columns=v)
        # append dfss list with dfs for each parameter
        dfss.append(new_df)
        

    # concatenate with outer join to keep all record IDs for each var
        # will fill patients with no values for certain records w NaN
    ext_var = reduce(lambda x, y: pd.merge(x, y, how='outer', on='RecordID'), dfss)
 
    # return dataframe into new variable
    return ext_var

In [14]:
df_extvars = extract_var(dftime_order, params, row_index)

In [15]:
print("A: {}".format(df_extvars.shape))

A: (3997, 334)


In [16]:
dfstatic = dfstatic_order.astype('float')

In [17]:
df_extvars.dtypes


RecordID         float64
ALP_first        float64
ALP_last         float64
ALP_min          float64
ALP_max          float64
ALP_q1           float64
ALP_median       float64
ALP_q3           float64
ALP_mean         float64
ALP_count        float64
ALT_first        float64
ALT_last         float64
ALT_min          float64
ALT_max          float64
ALT_q1           float64
ALT_median       float64
ALT_q3           float64
ALT_mean         float64
ALT_count        float64
AST_first        float64
AST_last         float64
AST_min          float64
AST_max          float64
AST_q1           float64
AST_median       float64
AST_q3           float64
AST_mean         float64
AST_count        float64
Albumin_first    float64
Albumin_last     float64
                  ...   
Urine_q3         float64
Urine_mean       float64
Urine_count      float64
WBC_first        float64
WBC_last         float64
WBC_min          float64
WBC_max          float64
WBC_q1           float64
WBC_median       float64


In [18]:
df_extvar = df_extvars.sort_values(by=['RecordID'])

In [19]:
final_stat = pd.merge(dfstatic, df_extvar, how='outer', on='RecordID')

In [20]:
final_stat.to_csv('pred_ready.csv')